In [ ]:
# from Pianki.Analiza_pianek import pda
# from Pianki.Analiza_pianek.przygotowanie_danych import aktualizuj_saldo, aktualizuj_zamowienia
# aktualizuj_saldo()
# aktualizuj_zamowienia(pda, "2024-08-12")


In [2]:
from Modele_db.modele_db import *
from Pianki.Analiza_pianek.funkcje_analizy_pianek import *

# pd.set_option()

In [17]:
from Modele_db.modele_db import ZAM_PIANKI, OWATY, session
import pandas as pd

owaty_w_toku = pd.DataFrame(session.query(ZAM_PIANKI.nr_partii, ZAM_PIANKI.opis, ZAM_PIANKI.ile_zam).filter(
                    ZAM_PIANKI.owaty_kompletacja == None, ZAM_PIANKI.data_zakonczenia == None).all(),
            columns=["NR_PARTII", "OPIS", "ILE_ZAM"]).merge(
    pd.pivot_table(pd.DataFrame(session.query(OWATY.OPIS, OWATY.TYP_OWATY, OWATY.ZUZYCIE).all()), values="ZUZYCIE", columns="TYP_OWATY", index="OPIS", fill_value=0),
    how="left",
    on="OPIS"
).dropna(axis=0)

owaty_w_toku["L1"] = owaty_w_toku.L1 * owaty_w_toku.ILE_ZAM  
owaty_w_toku["O1"] = owaty_w_toku.O1 * owaty_w_toku.ILE_ZAM  
owaty_w_toku["O2"] = owaty_w_toku.O2 * owaty_w_toku.ILE_ZAM  
owaty_w_toku["O3"] = owaty_w_toku.O3 * owaty_w_toku.ILE_ZAM  
owaty_w_toku["W3"] = owaty_w_toku.W3 * owaty_w_toku.ILE_ZAM  
           

ozn_owat = {                  #g/m2, szer, mb
    "O1": ["B/16/150 (1.6x50)", 150, 1.6, 50, "zielona", "73.115.00001"],
    "O2": ["B/16/200 (1.2x40)", 200, 1.2, 40, "niebieska", "73.120.00001"],
    "O3": ["B/16/200 (1.6x40)", 200, 1.6, 40, "czerwona", "73.120.00002"],
}

owaty_podsumowanie = owaty_w_toku.iloc[:,3:].sum()
owaty_Rolki = pd.Series({"L1": 60, 
                         "O1": 50, 
                         "O2": 40, 
                         "O3": 40, 
                         "W3":1000})

owaty = pd.DataFrame([owaty_podsumowanie, owaty_Rolki]).T
owaty.columns = ["ZAPOTRZ", "ROLKI"]
owaty["ILE ROLEK ZPOTRZ"] = owaty.ZAPOTRZ / owaty.ROLKI
owaty.astype(int)


,ZAPOTRZ,ROLKI,ILE ROLEK ZPOTRZ
L1,134,60,2
O1,1274,50,25
O2,558,40,13
O3,2421,40,60
W3,737,1000,0


In [15]:
from Modele_db.modele_db import session, ZAM_PIANKI, KOMPLETY_PIANEK
from sqlalchemy import func
from datetime import datetime as dt


session.query(func.sum(ZAM_PIANKI.ile_zam * KOMPLETY_PIANEK.obj)).join(KOMPLETY_PIANEK, KOMPLETY_PIANEK.kod == ZAM_PIANKI.kod).filter(
# session.query(ZAM_PIANKI.opis, ZAM_PIANKI.ile_zam, KOMPLETY_PIANEK.obj, ZAM_PIANKI.ile_zam * KOMPLETY_PIANEK.obj).join(KOMPLETY_PIANEK, KOMPLETY_PIANEK.kod == ZAM_PIANKI.kod).filter(
# session.query(ZAM_PIANKI.opis, ZAM_PIANKI.ile_zam).filter(
    ZAM_PIANKI.data_zakonczenia.between(dt(2024,11,3), dt(2024,11,10))).all()


[(306.335,)]

In [16]:
Ogolna_analiza_objetosci()
print("---\nZAPEŁNIENIE MAGAZYNU")
Ogolna_analiza_objetosci("podsum_prc")

MIN_obj:                     1764
MAX_obj:                     3519
ZAMOWIONE_obj:               792
CZEKA_NA_SPAKOWANIE_obj:     252
CZESCIOWO_DOSTARCZONE_obj:   0
SALDO_obj:                   2628
WOLNE_obj:                   2237
WOLNE_NIE_SPAK_obj:          2488
DO_ZAM_obj:                  485
---
ZAPEŁNIENIE MAGAZYNU
SALDO / MAX: 74.7%
WOLNE / MAX: 63.6%
WOLNE_NIE_SPAK / MAX: 70.7%
ZAPEŁNIENIE MAG PO ZDJECIU 3 PACZEK RAZEM Z ZAMOWIENIAMI: 93.2%


In [ ]:
br = Braki(WOLNE="SALDO")[0] #USUNAC PROBLEM Z DUBLUJACYMI SIE ZAMOWIANIAMI(JEDNO DOSTARCZONE DRUGIE W DRODZE TWORZĄ SIĘ DWIE POZYCJE)
br.sort_values(by=["GRUPA", "OPIS"]).drop_duplicates("OPIS")
# br.to_excel("BRAKI.xlsx")

In [6]:
Zagrozone(WOLNE="SALDO")
zgr = Zagrozone(False,WOLNE="SALDO")
# zgr
# zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)]
# zgr[(zgr.SALDO < zgr.MIN)]
zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)&(zgr.SALDO < zgr.MIN)]

WOLNE PONIZEJ MIN: 94 POZYCJE
WOLNE PONIZEJ MIN NIE ZAMOWIONE: 22 POZYCJE
SALDO PONIZEJ MIN: 66 POZYCJE
SALDO PONIZEJ MIN NIE ZAMOWIONE: 11 POZYCJE


,OPIS,ZAMOWIONE,CZEKA_NA_SPAKOWANIE,SALDO,MIN,WOLNE_SALDO
7,AVANT 2,0.0,0.0,3.0,5,0.0
94,"EXTREME 1,5s",0.0,0.0,0.0,8,0.0
97,"EXTREME [1,5s",0.0,0.0,0.0,8,0.0
107,EXTREME P80x105,0.0,0.0,0.0,8,0.0
132,HUDSON [LH,0.0,0.0,6.0,8,6.0
197,RITZ 4,0.0,0.0,0.0,4,0.0
219,SPECTRA ][,0.0,0.0,34.0,50,30.0
233,STONE P106x106,0.0,0.0,7.0,10,6.0
248,WILLOW P106x82,0.0,0.0,12.0,15,10.0
279,MISTRAL 3,0.0,0.0,9.0,10,9.0


In [19]:
#IMPORT ANAMLIZY PIANEK
from Pianki.Analiza_pianek.instrukcje_zamawiana import instrukcja_zamawiania_pianpol as izp
from Pianki.Analiza_pianek.Podsumowanie_analizy_pianek import Podsumowanie_analizy_pianek

arp = Podsumowanie_analizy_pianek(izp)
# arc = Podsumowanie_analizy_pianek(izc)

VOL 0!! ELI_MIS


In [20]:
arp

OBJ_CIECH        0.00
OBJ_VITA        72.12
OBJ_PIANPOL    236.77
RAZEM          308.89
---------
SALDO	WOLNE	MAX
2628	2237	3519
75%	64%	93%

In [22]:
analiza[analiza.OPIS.str.contains("WIL")][['KOD', 'OPIS', 'MAX','SALDO',
       'ZLECENIA', 'WST', 'CZEKA_NA_SPAKOWANIE', 'ZAMOWIONE',
       'CZESIOWO_DOSTARCZONE', 'MIN', 'SUMA_ZLEC',
       'SALDO_Z_NIE_SPAK', 'WOLNE_SALDO', 'WOLNE_NIE_SPAK', 'DO_ZAM_SZT',
       ]]

,KOD,OPIS,MAX,SALDO,ZLECENIA,WST,CZEKA_NA_SPAKOWANIE,ZAMOWIONE,CZESIOWO_DOSTARCZONE,MIN,SUMA_ZLEC,SALDO_Z_NIE_SPAK,WOLNE_SALDO,WOLNE_NIE_SPAK,DO_ZAM_SZT
240,16.130.10200.60,WILLOW 1,70.0,54.0,13.0,0.0,0.0,0.0,0,35,13.0,54.0,41.0,41.0,29.0
241,16.130.15200.60,"WILLOW 1,5",120.0,105.0,12.0,0.0,0.0,0.0,0,60,12.0,105.0,93.0,93.0,27.0
242,16.130.25200.60,"WILLOW 2,5",20.0,22.0,2.0,0.0,0.0,0.0,0,10,2.0,22.0,20.0,20.0,0.0
243,16.130.40200.60,WILLOW 4,30.0,26.0,3.0,0.0,0.0,0.0,0,15,3.0,26.0,23.0,23.0,7.0
244,16.130.66200.60,WILLOW N,23.0,16.0,0.0,0.0,0.0,0.0,0,12,0.0,16.0,16.0,16.0,7.0
245,16.130.63100.60,WILLOW [L,35.0,23.0,7.0,0.0,0.0,0.0,0,18,7.0,23.0,16.0,16.0,19.0
246,16.130.65000.60,WILLOW [L2Hz,40.0,41.0,7.0,0.0,0.0,0.0,0,20,7.0,41.0,34.0,34.0,6.0
247,16.130.77300.60,WILLOW P106x106,10.0,10.0,0.0,0.0,0.0,0.0,0,5,0.0,10.0,10.0,10.0,0.0
248,16.130.77400.60,WILLOW P106x82,30.0,12.0,2.0,0.0,0.0,0.0,0,15,2.0,12.0,10.0,10.0,20.0
249,16.130.73900.60,WILLOW P82x82,15.0,9.0,2.0,0.0,0.0,0.0,0,8,2.0,9.0,7.0,7.0,8.0


In [23]:
(arp > 1).query("GRUPA < 4")

,GRUPA,MODEL,MIN_obj,MAX_obj,SALDO_obj,WOLNE_obj,ZAMOWIONE_NIE_PRZYJETE_obj,BRYL_DO_ZAMOWIENIA,WSPL_DO_ZAM,OBJ_CIECH,OBJ_VITA,OBJ_PIANPOL
1,1,ELIXIR,27.0,54.0,29.0,25.0,21.0,57.0,14.0,0,0.00,14.67
2,1,STONE,220.0,440.0,272.0,218.0,183.0,97.0,11.0,0,27.90,6.01
3,1,HORIZON,249.0,498.0,448.0,398.0,68.0,162.0,10.0,0,27.48,5.55
0,1,AVANT,224.0,447.0,385.0,294.0,151.0,79.0,7.0,0,0.00,17.86
8,2,WILLOW,101.0,200.0,180.0,154.0,0.0,123.0,26.0,0,0.00,38.05
6,2,MAXWELL,56.0,112.0,82.0,69.0,27.0,32.0,23.0,0,0.00,15.57
5,2,DIVA,63.0,126.0,78.0,65.0,46.0,51.0,16.0,0,0.00,12.77
4,2,AMALFI,127.0,254.0,199.0,178.0,65.0,40.0,8.0,0,0.00,14.47
7,2,OXYGEN,42.0,85.0,61.0,51.0,35.0,12.0,7.0,0,0.00,4.50
9,2,REVERSO,101.0,202.0,142.0,115.0,85.0,23.0,4.0,0,5.39,1.07


In [ ]:
arp.Tabela_podsumowania_analizy#.query("GRUPA == 4")

In [31]:
arp["STONE"].ar


,OPIS,ZAMOWIONE,CZESIOWO_DOSTARCZONE,CZEKA_NA_SPAKOWANIE,SALDO,WOLNE_SALDO,DO_ZAM_SZT,MIN,MAX,SALDO_Z_NIE_SPAK,WOLNE_NIE_SPAK,SUMA_ZLEC,33/01,34/01,35/01,WST
222,"STONE 1,5",50,0,0,251,221,0,135,270,251,221,30,9,5,16,0
223,"STONE 2,5",10,0,0,34,23,12,22,45,34,23,11,4,3,4,0
224,STONE 3,20,0,0,36,31,9,30,60,36,31,5,2,1,2,0
225,STONE 4,20,0,0,15,9,6,18,35,15,9,6,1,1,4,0
226,STONE 5,0,0,0,24,23,0,10,20,24,23,1,0,0,1,0
227,STONE ON,70,0,0,26,12,18,50,100,26,12,14,3,2,9,0
228,STONE NW140x150,20,0,65,30,21,4,55,110,95,86,9,3,2,4,0
229,STONE NW140x150 LEN,0,0,0,59,55,0,28,55,59,55,4,2,1,1,0
230,STONE WN150x140 LEN,40,0,0,48,43,0,28,55,48,43,5,1,1,3,0
231,STONE [LH,0,0,0,26,22,3,12,25,26,22,4,3,0,1,0


In [ ]:
"sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db"

